Advanced implementation of MLEnd Code

In [22]:
#required dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa

import zipfile
import sklearn
import random

In [23]:
sample_path = '/home/jovyan/ML/DATA/DATA_unzip_full/*.wav'
files = glob.glob(sample_path)
len(files)

2500

In [24]:
MLENDLS_df = pd.read_csv('/home/jovyan/ML/git-code/MLEndLS_final.csv').set_index('file_id') 

In [25]:
MLENDLS_df['area'].value_counts()

campus        823
westend       666
british       404
Euston        234
southbank     229
kensington    144
Name: area, dtype: int64

In [20]:
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [5]:
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    

    
    try:
        
        fs = None # if None, fs would be 22050 #fr = sr = sampling rate
        x, fs = librosa.load(file,sr=fs)

        if scale_audio: 
            #x = x/np.max(np.abs(x))
            x = sklearn.preprocessing.minmax_scale(x, axis=0)
            x = librosa.effects.preemphasis(x)

        f0, voiced_flag = getPitch(x,fs,winLen=0.02)

        #yi = labels_file.loc[fileID]['in_out']=='indoor'
        y1 = labels_file.loc[fileID]['in_out']
        y2 = labels_file.loc[fileID]['area']
        y3 = labels_file.loc[fileID]['spot']
        yi = [ y1, y2, y3 ]

        power = np.sum(x**2)/len(x)
        pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
        pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
        voiced_fr = np.mean(voiced_flag)

        mfccs = librosa.feature.mfcc(y = x, sr=fs)
        spectral_centroids = librosa.feature.spectral_centroid(y = x, sr=fs)[0]

        rmse = librosa.feature.rms(y=x)
        chroma_stft = librosa.feature.chroma_stft(y=x, sr=fs)
        spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=fs)
        rolloff = librosa.feature.spectral_rolloff(y=x, sr=fs)
        zcr = librosa.feature.zero_crossing_rate(x)

        xi = [power,pitch_mean,pitch_std,voiced_fr, np.mean(np.mean(mfccs.T,axis=0)), np.mean(spectral_centroids), np.mean(rmse), np.mean(chroma_stft), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]
        X.append(xi)
        y.append(yi)
        
    except:
        print('issue in file index -->' + str(file))
      
    

  return np.array(X),np.array(y)

In [6]:
sample_file = random.sample(files, 500)

In [7]:
X,y = getXy(sample_file, labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)

  1%|▏         | 7/500 [00:24<23:35,  2.87s/it]  /opt/conda/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
  3%|▎         | 17/500 [01:14<33:03,  4.11s/it]  /opt/conda/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
  4%|▍         | 20/500 [01:37<58:09,  7.27s/it]/opt/conda/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
  4%|▍         | 21/500 [01:51<1:13:10,  9.17s/it]/opt/conda/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
  5%|▌         | 26/500 [02:26<53:04,  6.72s/it]  /opt/conda/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to est

In [26]:
print('The shape of X is', X.shape) 
print('The shape of y is', y.shape)

The shape of X is (500, 11)
The shape of y is (500, 3)


In [11]:
df1 = pd.DataFrame.from_records(X)
df1

,0,1,2,3,4,5,6,7,8,9,10
0,0.000375,131.176203,14.302356,0.025733,-13.329107,3059.435736,0.017930,0.777882,3109.872941,6450.418947,0.043074
1,0.000331,0.000000,0.000000,0.000000,-12.342485,3559.897865,0.017945,0.680987,3390.693027,7563.722072,0.018859
2,0.000186,175.941351,27.252490,0.057047,-15.457408,2352.233607,0.013413,0.736039,3207.815917,5742.307932,0.003223
3,0.000287,309.914699,142.885866,0.085670,-17.621130,3329.585607,0.016475,0.727327,4092.946321,7197.556997,0.015092
4,0.000196,212.983535,53.891787,0.063467,-16.661283,2897.988063,0.013749,0.676671,3360.657703,5913.497575,0.009024
...,...,...,...,...,...,...,...,...,...,...,...
495,0.000261,0.000000,0.000000,0.000000,-14.940593,3592.805387,0.015756,0.678571,3280.529532,6879.689450,0.031320
496,0.000244,320.124468,8.893479,0.029499,-13.765862,4511.764935,0.015450,0.523873,4972.494530,9925.377446,0.031623
497,0.000203,159.385457,13.077490,0.042895,-15.161199,7131.167778,0.013996,0.750911,5023.997690,13233.885213,0.066853
498,0.000264,212.369938,3.307037,0.006360,-16.526691,1803.725438,0.016202,0.896750,2832.025479,4810.068794,0.001093


In [12]:
df2 = pd.DataFrame.from_records(y)
df2

,0,1,2
0,outdoor,british,street
1,outdoor,british,street
2,outdoor,Euston,forecourt
3,indoor,british,greatcourt
4,outdoor,westend,trafalgar
...,...,...,...
495,outdoor,southbank,skate
496,indoor,campus,ground
497,outdoor,campus,canal
498,outdoor,campus,square


In [14]:
df3 = df1.join(df2, lsuffix="_x", rsuffix="_y")
df3

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,0_y,1_y,2_y
0,0.000375,131.176203,14.302356,0.025733,-13.329107,3059.435736,0.017930,0.777882,3109.872941,6450.418947,0.043074,outdoor,british,street
1,0.000331,0.000000,0.000000,0.000000,-12.342485,3559.897865,0.017945,0.680987,3390.693027,7563.722072,0.018859,outdoor,british,street
2,0.000186,175.941351,27.252490,0.057047,-15.457408,2352.233607,0.013413,0.736039,3207.815917,5742.307932,0.003223,outdoor,Euston,forecourt
3,0.000287,309.914699,142.885866,0.085670,-17.621130,3329.585607,0.016475,0.727327,4092.946321,7197.556997,0.015092,indoor,british,greatcourt
4,0.000196,212.983535,53.891787,0.063467,-16.661283,2897.988063,0.013749,0.676671,3360.657703,5913.497575,0.009024,outdoor,westend,trafalgar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000261,0.000000,0.000000,0.000000,-14.940593,3592.805387,0.015756,0.678571,3280.529532,6879.689450,0.031320,outdoor,southbank,skate
496,0.000244,320.124468,8.893479,0.029499,-13.765862,4511.764935,0.015450,0.523873,4972.494530,9925.377446,0.031623,indoor,campus,ground
497,0.000203,159.385457,13.077490,0.042895,-15.161199,7131.167778,0.013996,0.750911,5023.997690,13233.885213,0.066853,outdoor,campus,canal
498,0.000264,212.369938,3.307037,0.006360,-16.526691,1803.725438,0.016202,0.896750,2832.025479,4810.068794,0.001093,outdoor,campus,square


In [16]:
df3.to_csv('feature-set-3-adv.csv')

In [27]:
df_adv = pd.read_csv('/home/jovyan/ML/git-code/feature-set-3-adv.csv').set_index('id') 
df_adv

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,0_y,1_y,2_y
id,,,,,,,,,,,,,,
0,0.000375,131.176203,14.302356,0.025733,-13.329107,3059.435736,0.017930,0.777882,3109.872941,6450.418947,0.043074,outdoor,british,street
1,0.000331,0.000000,0.000000,0.000000,-12.342485,3559.897865,0.017945,0.680987,3390.693027,7563.722072,0.018859,outdoor,british,street
2,0.000186,175.941351,27.252490,0.057047,-15.457408,2352.233607,0.013413,0.736039,3207.815917,5742.307932,0.003223,outdoor,Euston,forecourt
3,0.000287,309.914699,142.885866,0.085670,-17.621130,3329.585607,0.016475,0.727327,4092.946321,7197.556997,0.015092,indoor,british,greatcourt
4,0.000196,212.983535,53.891787,0.063467,-16.661283,2897.988063,0.013749,0.676671,3360.657703,5913.497575,0.009024,outdoor,westend,trafalgar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000261,0.000000,0.000000,0.000000,-14.940593,3592.805387,0.015756,0.678571,3280.529532,6879.689450,0.031320,outdoor,southbank,skate
496,0.000244,320.124468,8.893479,0.029499,-13.765862,4511.764935,0.015450,0.523873,4972.494530,9925.377446,0.031623,indoor,campus,ground
497,0.000203,159.385457,13.077490,0.042895,-15.161199,7131.167778,0.013996,0.750911,5023.997690,13233.885213,0.066853,outdoor,campus,canal


In [28]:
df_adv['1_y'].value_counts()

campus        155
westend       126
british        90
Euston         56
southbank      48
kensington     25
Name: 1_y, dtype: int64

In [29]:
options = ['campus', 'westend','british']
df_1 = df_adv.loc[df_adv['1_y'].isin(options)]
df_1

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,0_y,1_y,2_y
id,,,,,,,,,,,,,,
0,0.000375,131.176203,14.302356,0.025733,-13.329107,3059.435736,0.017930,0.777882,3109.872941,6450.418947,0.043074,outdoor,british,street
1,0.000331,0.000000,0.000000,0.000000,-12.342485,3559.897865,0.017945,0.680987,3390.693027,7563.722072,0.018859,outdoor,british,street
3,0.000287,309.914699,142.885866,0.085670,-17.621130,3329.585607,0.016475,0.727327,4092.946321,7197.556997,0.015092,indoor,british,greatcourt
4,0.000196,212.983535,53.891787,0.063467,-16.661283,2897.988063,0.013749,0.676671,3360.657703,5913.497575,0.009024,outdoor,westend,trafalgar
5,0.000359,223.066313,16.486602,0.064516,-13.796000,3793.996894,0.018799,0.659943,3892.835085,7537.940871,0.007134,outdoor,campus,library
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0.000218,112.490473,13.922397,0.062295,-14.032511,2363.327802,0.014402,0.790075,3171.482534,5151.165791,0.006849,outdoor,british,square
496,0.000244,320.124468,8.893479,0.029499,-13.765862,4511.764935,0.015450,0.523873,4972.494530,9925.377446,0.031623,indoor,campus,ground
497,0.000203,159.385457,13.077490,0.042895,-15.161199,7131.167778,0.013996,0.750911,5023.997690,13233.885213,0.066853,outdoor,campus,canal


In [119]:
X = df_1.drop(['1_y'], axis=1)
print(X['0_y'].value_counts())

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le.fit(X['0_y'])
print(list(le.classes_))
X['0_y'] = le.transform(X['0_y'])

le.fit(X['2_y'])
print(list(le.classes_))
X['2_y'] = le.transform(X['2_y'])

X

indoor     189
outdoor    182
Name: 0_y, dtype: int64
['indoor', 'outdoor']
['canal', 'charing', 'curve', 'forecourt', 'greatcourt', 'ground', 'leicester', 'library', 'market', 'national', 'piazza', 'reception', 'room12', 'square', 'street', 'trafalgar']


,0_x,1_x,2_x,3,4,5,6,7,8,9,10,0_y,2_y
id,,,,,,,,,,,,,
0,0.000375,131.176203,14.302356,0.025733,-13.329107,3059.435736,0.017930,0.777882,3109.872941,6450.418947,0.043074,1,14
1,0.000331,0.000000,0.000000,0.000000,-12.342485,3559.897865,0.017945,0.680987,3390.693027,7563.722072,0.018859,1,14
3,0.000287,309.914699,142.885866,0.085670,-17.621130,3329.585607,0.016475,0.727327,4092.946321,7197.556997,0.015092,0,4
4,0.000196,212.983535,53.891787,0.063467,-16.661283,2897.988063,0.013749,0.676671,3360.657703,5913.497575,0.009024,1,15
5,0.000359,223.066313,16.486602,0.064516,-13.796000,3793.996894,0.018799,0.659943,3892.835085,7537.940871,0.007134,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0.000218,112.490473,13.922397,0.062295,-14.032511,2363.327802,0.014402,0.790075,3171.482534,5151.165791,0.006849,1,13
496,0.000244,320.124468,8.893479,0.029499,-13.765862,4511.764935,0.015450,0.523873,4972.494530,9925.377446,0.031623,0,5
497,0.000203,159.385457,13.077490,0.042895,-15.161199,7131.167778,0.013996,0.750911,5023.997690,13233.885213,0.066853,1,0


In [120]:
# X = df_1.drop(['1_y', '0_y', '2_y'], axis=1)
y = df_1['1_y']
y

id
0      british
1      british
3      british
4      westend
5       campus
        ...   
494    british
496     campus
497     campus
498     campus
499     campus
Name: 1_y, Length: 371, dtype: object

In [121]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)

list(le.classes_)
le.transform(['british', 'campus', 'westend'])

array([0, 1, 2])

In [122]:
y_n = le.transform(y)
y_n.shape

(371,)

In [123]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y_n,test_size=0.3)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((259, 13), (112, 13), (259,), (112,))

In [ ]:
X_train

In [125]:
from keras import layers
from keras import layers
import keras
from keras.models import Sequential

model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [126]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier = model.fit(X_train, y_train, epochs = 100, batch_size = 128)

Epoch 1/100
3/3 [==============================] - 0s 8ms/step - loss: 306.4663 - accuracy: 0.2934
Epoch 2/100
3/3 [==============================] - 0s 6ms/step - loss: 189.7059 - accuracy: 0.4363
Epoch 3/100
3/3 [==============================] - 0s 6ms/step - loss: 213.8361 - accuracy: 0.3320
Epoch 4/100
3/3 [==============================] - 0s 5ms/step - loss: 102.7799 - accuracy: 0.2664
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 165.7784 - accuracy: 0.4054
Epoch 6/100
3/3 [==============================] - 0s 5ms/step - loss: 91.6026 - accuracy: 0.4170
Epoch 7/100
3/3 [==============================] - 0s 5ms/step - loss: 110.3555 - accuracy: 0.3320
Epoch 8/100
3/3 [==============================] - 0s 6ms/step - loss: 132.0307 - accuracy: 0.2510
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 47.3883 - accuracy: 0.3668
Epoch 10/100
3/3 [==============================] - 0s 6ms/step - loss: 125.1708 - accuracy: 0.4363
Epoch 11/10

In [127]:
test_loss, test_acc = model.evaluate(X_val,y_val)

4/4 [==============================] - 0s 4ms/step - loss: 18.0888 - accuracy: 0.2857


In [128]:
from sklearn import svm

model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)
print(y_train)
print(yt_p)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

[0 1 2 2 1 0 0 1 0 1 2 1 0 2 0 1 2 1 1 0 1 0 2 1 0 1 2 1 1 1 1 1 2 1 2 1 2
 0 0 0 0 2 2 1 1 1 0 1 1 2 1 2 1 0 1 2 0 2 1 2 1 2 2 1 2 1 0 1 2 1 1 2 2 2
 1 1 2 1 0 0 2 2 1 0 1 1 1 1 1 2 2 2 1 1 1 2 1 2 2 0 2 0 2 1 2 1 0 2 1 1 0
 2 0 1 1 1 2 2 0 2 1 1 0 2 0 0 2 1 2 2 1 0 1 2 2 2 0 1 1 0 1 0 1 0 1 1 2 1
 2 2 0 2 1 2 1 1 2 2 2 2 1 1 2 0 1 0 1 2 2 1 1 2 1 1 1 0 1 1 0 1 2 2 2 2 1
 1 0 1 1 1 0 2 2 0 0 1 2 1 1 1 0 2 1 2 1 1 2 1 2 2 1 1 0 0 0 1 2 0 0 2 0 2
 0 0 0 1 0 2 1 1 0 0 1 2 2 1 2 0 1 1 0 0 1 1 1 1 1 1 2 1 2 1 1 2 0 2 1 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1

In [129]:
mean = X_train.mean(0)
sd =  X_train.std(0)

X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

model  = svm.SVC(C=1,gamma=2)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print(yv_p)
print(y_val)
print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[1 0 0 2 1 0 1 2 0 0 2 0 2 1 2 2 0 1 2 2 0 0 2 1 1 0 0 2 2 1 1 2 2 2 0 2 1
 1 2 1 1 1 2 1 1 0 1 2 1 1 1 1 2 1 1 1 1 1 1 2 1 2 0 1 2 2 2 0 0 1 2 1 1 1
 2 1 0 0 1 1 1 1 2 2 0 0 2 2 0 2 2 2 0 0 0 0 2 0 2 1 0 2 2 1 0 0 2 2 1 0 2
 1]
Training Accuracy 1.0
Validation  Accuracy 0.375
